<a href="https://colab.research.google.com/github/maithstartup/Practical-Machine-Learning-Bootcamp/blob/master/Decision_Tree_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***

#  Introduction to Decision Trees



Decision Trees are an important type of algorithm for predictive modeling machine learning.

The classical decision tree algorithms have been around for decades and modern variations like random forest are among the most powerful techniques available.

Classification and Regression Trees or `CART` for short is a term introduced by `Leo Breiman` to refer to Decision Tree algorithms that can be used for classification or regression predictive modeling problems.

Classically, this algorithm is referred to as “`decision trees`”, but on some platforms like R they are referred to by the more modern term CART.

The `CART` algorithm provides a foundation for important algorithms like `bagged decision trees`, `random forest` and `boosted decision trees`.

### CART Model Representation
The representation for the CART model is a binary tree.

This is your binary tree from algorithms and data structures, nothing too fancy. Each root node represents a single input variable (x) and a split point on that variable (assuming the variable is numeric).

The leaf nodes of the tree contain an output variable (y) which is used to make a prediction.

Given a new input, the tree is traversed by evaluating the specific input started at the root node of the tree.



# Decision Tree Implementation in python

We will use Decision Tree in Predicting the attrition of your valuable employees.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

# Problem Statement

The key to success in any organization is attracting and retaining top talent. Our task is to classify which employess are likely to leave the compay and determine which factors keep employees at my company and which prompt others to leave. 

In [0]:
df = pd.read_csv("HR-Employee-Attrition.csv")         #change the directory to the location of the file 
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


## 1. Exploratory Data Analysis

The below table shows us that the dataset consists  of 1470 rows and 35 columns. It also gives us information whether the features are categorical or numerical and indicates there or no missing values in the dataset.

In [0]:
print(df.shape)
df.info()

In [0]:
pd.set_option("display.float_format", "{:.2f}".format)
df.describe() #gives the descriptive statistics of the data

In [0]:
df.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis="columns", inplace=True)

In [0]:
categorical_col = []
for column in df.columns:
    if df[column].dtype == object and len(df[column].unique()) <= 50:
        categorical_col.append(column)
        print(f"{column} : {df[column].unique()}")
        print("====================================")

In [0]:
df['Attrition'] = df.Attrition.astype("category").cat.codes

## 2. Data Visualisation

In [0]:
df.Attrition.value_counts()

In [0]:
# Visulazing the distibution of the data for every feature
df.hist(edgecolor='black', linewidth=1.2, figsize=(20, 20));

In [0]:
# Plotting how every feature correlate with the "target"
sns.set(font_scale=1.2)
plt.figure(figsize=(30, 30))

for i, column in enumerate(categorical_col, 1):
    plt.subplot(3, 3, i)
    g = sns.barplot(x=f"{column}", y='Attrition', data=df)
    g.set_xticklabels(g.get_xticklabels(), rotation=90)
    plt.ylabel('Attrition Count')
    plt.xlabel(f'{column}')

**Conclusions:**

***
- `BusinessTravel` : The workers who travel alot are more likely to quit then other employees.

- `Department` : The worker in `Research & Development` are more likely to stay then the workers on other departement.

- `EducationField` : The workers with `Human Resources` and `Technical Degree` are more likely to quit then employees from other fields of educations.

- `Gender` : The `Male` are more likely to quit.

- `JobRole` : The workers in `Laboratory Technician`, `Sales Representative`, and `Human Resources` are more likely to quit the workers in other positions.

- `MaritalStatus` : The workers who have `Single` marital status are more likely to quit the `Married`, and `Divorced`.

- `OverTime` : The workers who work more hours are likely to quit then others.

*** 

## 3. Data Processing

In [0]:
categorical_col.remove('Attrition')

In [0]:
# Transform categorical data into dummies
# categorical_col.remove("Attrition")
# data = pd.get_dummies(df, columns=categorical_col)
# data.info()
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
for column in categorical_col:
    df[column] = label.fit_transform(df[column])

The label encoder converts all categorical variables into numerical variables as each class corresponding to a number for training the model

In [0]:
X = df.drop('Attrition', axis=1)
y = df.Attrition

## 4. Applying machine learning algorithms

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        print("Train Result:\n===========================================")
        print(f"accuracy score: {accuracy_score(y_train, pred):.4f}\n")
        print(f"Classification Report: \n \tPrecision: {precision_score(y_train, pred)}\n\tRecall Score: {recall_score(y_train, pred)}\n\tF1 score: {f1_score(y_train, pred)}\n")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, clf.predict(X_train))}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        print("Test Result:\n===========================================")        
        print(f"accuracy score: {accuracy_score(y_test, pred)}\n")
        print(f"Classification Report: \n \tPrecision: {precision_score(y_test, pred)}\n\tRecall Score: {recall_score(y_test, pred)}\n\tF1 score: {f1_score(y_test, pred)}\n")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 5 Decision Tree Classifier Parameters

- `criterion`: The function to measure the quality of a split. Supported criteria are "`gini`" for the Gini impurity and "`entropy`" for the information gain.
***
- `splitter`: The strategy used to choose the split at each node. Supported strategies are "`best`" to choose the best split and "`random`" to choose the best random split.
***
- `max_depth`: The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than `min_samples_split` samples.
***
- `min_samples_split`: The minimum number of samples required to split an internal node.
***
- `min_samples_leaf`: The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least ``min_samples_leaf`` training samples in each of the left and right branches.  This may have the effect of smoothing the model, especially in regression.
***
- `min_weight_fraction_leaf`: The minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node. Samples have equal weight when sample_weight is not provided.
***
- `max_features`: The number of features to consider when looking for the best split.
***
- `max_leaf_nodes`: Grow a tree with ``max_leaf_nodes`` in best-first fashion. Best nodes are defined as relative reduction in impurity. If None then unlimited number of leaf nodes.
***
- `min_impurity_decrease`: A node will be split if this split induces a decrease of the impurity greater than or equal to this value.
***
- `min_impurity_split`: Threshold for early stopping in tree growth. A node will split if its impurity is above the threshold, otherwise it is a leaf.

In [0]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

print_score(tree, X_train, y_train, X_test, y_test, train=True)
print_score(tree, X_train, y_train, X_test, y_test, train=False)

Confusion matrix and F1 score is used as a evaluation metric as the dataset is imbalanced. From the above results, it can be inferred that when we train using the default parameters of the DecisionTreeClassifier without restricting parameters like max_depth and min_samples_leaf the model overfits. This overfitting can be understood from the train accuracy and F1 score being 1 which shows that no point has been misclassified in training data. The very low F1 score of 0.28 indicates that the model performs very badly on test data. Our aim is to achieve a trade-off between train and test accuracy.

### Visualization of a tree

In [0]:
import graphviz
from sklearn.tree import export_graphviz
features = list(df.columns)
features.remove("Attrition")
classes=['Yes','No']
dot_data = export_graphviz(tree, out_file=None, 
                     feature_names=features,
                     class_names=classes,      
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)  
graph.view()
graph

The decision tree can be visualized using the graphviz library. At each node (decision) of the tree, we get the variable of splitting with the value, gini index of the samples of the dataset at that node, no of samples in each class and the majority class. A decision on which class (Attrition: Yes or No) a data point belongs to can be found by traversing through one branch of the tree.The above tree which has been trained with default parameters has a depth of 16. Higher the depth, more the chances of overfitting. The overfitting can be seen in the leaf nodes where the no of samples in each class is a very small number like one or two. This implies the dataset has an high risk of overfitting to the noisy points in the data.   

## 5.1 Decision Tree Classifier Hyper-Parameters

## Criterion

The time taken for the decision tree classifier for training and the accuracy with gini impurity as the criterion is given in the below code. The task is to change the criterion to entropy and find the accuracy and training time. Compare the results and write your inferences on it.

In [0]:
import time
t0 = time.time()
tree = DecisionTreeClassifier(random_state=42,max_depth = 3)
tree.fit(X_train, y_train)
print("Training time using gini criterion", round(time.time()-t0, 5), "s")
print('Accuracy using the defualt gini impurity criterion\n')
print_score(tree, X_train, y_train, X_test, y_test, train=False)

##################write your code here######################



## Splitter

In [0]:
t = time.time()
clf = DecisionTreeClassifier(random_state= 42,max_depth = 3, criterion = "entropy", splitter = 'best')
clf.fit(X_train,y_train)
print('Best Split running time...',time.time() - t)
print('Best Split accuracy...',clf.score(X_test,y_test))

t = time.time()
clf = DecisionTreeClassifier(random_state= 42,max_depth = 3, criterion = "entropy", splitter = 'random')
clf.fit(X_train,y_train)
print('Random Split running time...',time.time() - t)
print('Random Split accuracy...',clf.score(X_test,y_test))


Even though there is a drop in the accuracy while using random split, random split decreases the training time significantly compared to best split as it takes a random subset of features for computing the Information gain. 

## Max Depth
The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

In [0]:
test_score = []
train_score = []
for depth in range(20):
    clf = DecisionTreeClassifier(random_state= 42,max_depth = depth + 1)
    clf.fit(X_train,y_train)
    train_score.append(clf.score(X_train,y_train))
    test_score.append(clf.score(X_test,y_test))

plt.figure(figsize = (6,6))
plt.plot(range(20),train_score)
plt.plot(range(20), test_score)
plt.xlabel('Tree Depth')
plt.ylabel('Accuracy')
plt.legend(['Training set','Test set'])

At a depth of around 5, a good tradeoff between test and train accuracy is obtained. At a depth of 1 or 2, the tree is shallow and underfits.

The task is to build a decision tree with optimal depth as mentioned above and other suitable values for the parameters criterion and splitter. Visualize the classifier using export_graphviz() and write your inferences

In [0]:
##################write your code here######################










#graph.view()      #Uncomment text to view it as pdfgraph

## Max_Features

In [0]:
test_score = []
train_score = []
max_features = range(len(df.columns)-1)
for feat in max_features:
    clf = DecisionTreeClassifier(random_state= 42,max_features = feat + 1,max_depth=5,criterion = "entropy", splitter = 'best')
    clf.fit(X_train,y_train)
    train_score.append(clf.score(X_train,y_train))
    test_score.append(clf.score(X_test,y_test))

plt.figure(figsize = (6,6))
plt.plot(max_features,train_score)
plt.plot(max_features, test_score)
plt.xlabel('Max Features')
plt.ylabel('Accuracy')
plt.legend(['Training set','Test set'])

The max_features set to 3 gives a good accuracy on both the train and test set. This shows that comparing Information gain for just 3 features during a split can fit a really good model and also save a lot of computing time. But conclusive evidence on how many features is to be used for splitting can't be obtained from the graph as it is noisy.

# Min sample leaf
The minimum number of samples required to be at a leaf node: If int, then consider min_samples_leaf as the minimum number. If float, then min_samples_leaf is a percentage and ceil(min_samples_leaf * n_samples) are the minimum number of samples for each node.

The task is to find the optimal minimum number of samples required to be at a leaf node from the given min_sample_leaf values. Calculate the train and test scores of the model and plot the same

In [0]:
test_score = []
train_score = []
min_sample_leaf = np.arange(5,100,5)
##################write your code here######################





plt.figure(figsize = (8,8))
plt.plot(min_sample_leaf,train_score)
plt.plot(min_sample_leaf, test_score)
plt.xlabel('Min Sample Leaf')
plt.ylabel('Accuracy')
plt.legend(['Training set','Test set'])

The task is to build a decision tree with optimal min_samples_leaf =20 as mentioned above and other suitable values for the parameters criterion, splitter and depth. Visualize the classifier using export_graphviz() and write your inferences

In [0]:
##################write your code here######################

## Training Decision tree with optimal parameters

Let us check the accuracy and F1 scores of the above model with best fit parameters

In [0]:
tree = DecisionTreeClassifier(random_state=42,criterion = "entropy",max_depth = 5,min_samples_leaf =20, splitter = 'best')
tree.fit(X_train, y_train)

print_score(tree, X_train, y_train, X_test, y_test, train=True)
print_score(tree, X_train, y_train, X_test, y_test, train=False)

As the dataset is imbalanced, F1 score should also be considered while evaluating the model. Even though the train and test accuracies are high, F1 score values are less than 0.5.

To solve the imbalance in the dataset, set the class weight parameter to balanced in the above model.

In [0]:
##################write your code here######################

tree.fit(X_train, y_train)

print_score(tree, X_train, y_train, X_test, y_test, train=True)
print_score(tree, X_train, y_train, X_test, y_test, train=False)

F1 score can be increased by using the parameter class_weight = 'balanced' which penalizes mistakes in samples of class[i] with class_weight[i] instead of 1. This gives higher weights to the minority class

## 5.2. Decision Tree Classifier Hyperparameter tuning using GridSearchCV
GridSearchCV is a library function that is a member of sklearn’s model_selection package. It helps to loop through predefined hyperparameters and fit your estimator (model) on your training set. So, in the end, you can select the best parameters from the listed hyperparameters.In addition to that, you can specify the number of times for the cross-validation for each set of hyperparameters. The below model fits for all combinations of the 5 parameters given below with a 3-fold cross validation and the best estimators are obtained using Grid search.

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {
    "criterion":("gini", "entropy"), 
    "splitter":("best", "random"), 
    "max_depth":(list(range(1, 20))), 
    "min_samples_split":[2, 3, 4], 
    "min_samples_leaf":list(range(1, 20)), 
    "class_weight":(None,"balanced")
    #"max_features":(list(range(1,len(df.columns)))),
}


model = DecisionTreeClassifier(random_state=42)
grid_search_cv = GridSearchCV(model, params, scoring="accuracy", n_jobs=-1, verbose=1, cv=3)

grid_search_cv.fit(X_train, y_train)

In [0]:
 grid_search_cv.best_estimator_

The parameters for the best fit model suggested by Grid search is criterion = 'entropy', max_depth = 6, min_samples_leaf = 10.

In [0]:
tree = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=6, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [0]:
tree.fit(X_train, y_train)

In [0]:
print_score(tree, X_train, y_train, X_test, y_test, train=True)
print_score(tree, X_train, y_train, X_test, y_test, train=False)

The model obtained from the Grid search cross validation has a better F1 score than our model with optimal parameters.

### Visualization of a tree

In [0]:
features = list(df.columns)
classes=['Yes','No']
features.remove("Attrition")
dot_data = export_graphviz(tree, out_file=None, 
                     feature_names=features,   
                     class_names=classes,      
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)  
#graph.view()         #Uncomment text to view it as pdf
graph


## 6. Feature Selection for Decision trees

The task is to plot the feature importance of all the features in the dataset and find the best deatures which is useful in building the decision trees.

Hint : Use sklearn.tree.feature_importances_

In [0]:
#print(len(tree.feature_importances_))
#print(len(df.columns))
##################write your code here######################








## Additional Exercises

Upsample or downsample the dataset and find the best fit model using GridSearchCV

# 7. Summary
In this notebook we learned the following lessons:
- Decsion tree algorithms and the parameters of the algorithm.
- How to tune hyperparameters for  Decision trees.
- Balance your dataset before training to prevent the tree from being biased toward the classes that are dominant. 

  